# 8th_homework

## Стратегия выполнения проекта

Для выполнения курсового проекта используется двухуровневая рекомендательная система:
- на первом уровне модель <code>AlternatingLeastSquares</code> составляет список наиболее подходящих товаров для каждого клиента;
- на втором уровне модель <code>CatBoostClassifier</code>, используя признаки пользователей и товаров, для каждого клиента оставляет пять наиболее подходящих товаров.

Применение модели второго уровня значительно улучшает метрику <code>Precision@5</code>, однако **не для всех пользователей и товаров есть признаки**, необходимые для работы модели второго уровня. В случаях, когда о пользователях и товарах ничего не известно, используется модель первого уровня.

## Подготовка

### Импорт библиотек

In [1]:
import numpy as np
import pandas as pd

from src import utils
from src import recommenders
from src import metrics

from catboost import CatBoostClassifier

### Импорт данных

In [2]:
PATH_DATA = r'../2th lesson/retail_train.csv'
PATH_ITEM_FEATURES = r'../2th lesson/product.csv'
PATH_USER_FEATURES = r'../2th lesson/hh_demographic.csv'

PATH_RESULT = r'recommendations.csv'

### Глобальные настройки проекта

In [3]:
random_state_global = 0

## Модель первого уровня

### Загрузка данных

In [4]:
# Загрузка данных.
data = pd.read_csv(PATH_DATA)
item_features = pd.read_csv(PATH_ITEM_FEATURES)
user_features = pd.read_csv(PATH_USER_FEATURES)

# Обработка названий столбцов.
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features = item_features.rename(columns={'product_id': 'item_id'})
user_features = user_features.rename(columns={'household_key': 'user_id'})

In [5]:
# Проверка.
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [6]:
# Проверка.
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [7]:
# Проверка.
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


### Разбиение данных

In [8]:
# Разбиение данных.
data_train, data_test = utils.train_test_split(data, 'week_no', data['week_no'].max() - 9)
data_valid, data_test = utils.train_test_split(data_test, 'week_no', data_test['week_no'].max() - 3)

In [9]:
# Проверка: размер исходного набора данных.
data.shape

(2396804, 12)

In [10]:
# Проверка: размер обучающего набора данных.
data_train.shape

(2108779, 12)

In [11]:
# Удаление товаров, которых нет в обучающей выборке.
data_valid = data_valid[data_valid['user_id'].isin(data_train['user_id'])]
data_valid = data_valid[data_valid['item_id'].isin(data_train['item_id'])]

# Проверка: размер валидационного набора данных.
data_valid.shape

(165198, 12)

In [12]:
# Проверка: недели валидационного набора данных.
data_valid['week_no'].unique()

array([86, 87, 88, 89, 90, 91], dtype=int64)

In [13]:
# Удаление товаров, которых нет в обучающей выборке.
data_test = data_test[data_test['user_id'].isin(data_train['user_id'])]
data_test = data_test[data_test['item_id'].isin(data_train['item_id'])]

# Проверка: размер тестового набора данных.
data_test.shape

(114212, 12)

In [14]:
# Проверка: недели тестового набора данных.
data_test['week_no'].unique()

array([92, 93, 94, 95], dtype=int64)

### Обработка данных

In [15]:
# Фильтрация товаров.
data_train = utils.prefilter_items(data_train, 'item_id', 'quantity')

# Проверка.
data_train.tail()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2109568,856,40630539494,593,999999,1,1.99,372,-1.0,1831,85,0.0,0.0
2109569,856,40630539494,593,1120213,1,1.67,372,0.0,1831,85,0.0,0.0
2109570,856,40630539494,593,999999,1,5.69,372,-0.3,1831,85,0.0,0.0
2109571,856,40630539494,593,999999,1,10.99,372,-3.3,1831,85,0.0,0.0
2109572,856,40630539494,593,999999,1,0.89,372,0.0,1831,85,0.0,0.0


### Подготовка таблиц для результатов прогнозирования

In [16]:
# Подготовка тренировочной таблицы.
result_train = utils.prepare_result(data_train)

# Проверка.
result_train.head(2)

,user_id,actual
0,1,"[999999, 840361, 845307, 852014, 856942, 91267..."
1,2,"[854852, 930118, 1077555, 1098066, 999999, 556..."


In [17]:
# Подготовка валидационной таблицы.
result_valid = utils.prepare_result(data_valid)

# Проверка.
result_valid.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 875240, 87737..."
1,2,"[838136, 839656, 861272, 866211, 870791, 87391..."


In [18]:
# Подготовка тестовой таблицы.
result_test = utils.prepare_result(data_test)

# Проверка.
result_test.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### Построение модели первого уровня

#### Обучение модели

In [19]:
# Формирование таблицы метрик.
df_metrics = pd.DataFrame()

In [20]:
# Инициализация модели первого уровня.
model_1 = recommenders.MainRecommender(random_state=random_state_global)

In [21]:
%%time

# Обучения модели первого уровня.
model_1.fit(data_train)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 3.3 s


#### Прогноз на обучающей выборке

In [22]:
%%time

# Прогнозирование при помощи модели AlternatingLeastSquares на учебной выборке.
result_train['model_1'] = result_train['user_id'].apply(lambda user_id: model_1.predict_als(user_id, N=50))

# Проверка.
result_train.head(2)

Wall time: 6.46 s


,user_id,actual,model_1
0,1,"[999999, 840361, 845307, 852014, 856942, 91267...","[981760, 1005186, 911878, 962568, 826249, 8623..."
1,2,"[854852, 930118, 1077555, 1098066, 999999, 556...","[981760, 1005186, 844165, 911878, 826249, 8623..."


In [23]:
# Вычисление метрик на тестовой выборке.
df_metrics.loc['model_1', 'precision_train'] = metrics.precision_at_k(result_train['actual'], result_train['model_1'], K=5)

#### Прогноз на валидационной выборке

In [24]:
%%time

# Прогнозирование при помощи модели AlternatingLeastSquares на валидационной выборке.
result_valid['model_1'] = result_valid['user_id'].apply(lambda user_id: model_1.predict_als(user_id, N=50))

# Проверка.
result_valid.head(2)

Wall time: 6.01 s


,user_id,actual,model_1
0,1,"[853529, 865456, 867607, 872137, 875240, 87737...","[981760, 1005186, 911878, 962568, 826249, 8623..."
1,2,"[838136, 839656, 861272, 866211, 870791, 87391...","[981760, 1005186, 844165, 911878, 826249, 8623..."


In [25]:
# Вычисление метрик на валидационной выборке.
df_metrics.loc['model_1', 'precision_valid'] = metrics.precision_at_k(result_valid['actual'], result_train['model_1'], K=5)

#### Прогноз на тестовой выборке

In [26]:
%%time

# Прогнозирование при помощи модели AlternatingLeastSquares на тестовой выборке.
result_test['model_1'] = result_test['user_id'].apply(lambda user_id: model_1.predict_als(user_id, N=50))

# Проверка.
result_test.head(2)

Wall time: 5.75 s


,user_id,actual,model_1
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[981760, 1005186, 911878, 962568, 826249, 8623..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[981760, 6533889, 1053690, 929668, 844165, 965..."


In [27]:
# Вычисление метрик на валидационной выборке.
df_metrics.loc['model_1', 'precision_test'] = metrics.precision_at_k(result_test['actual'], result_test['model_1'], K=5)

#### Анализ метрик

In [28]:
# Вывод метрик.
df_metrics

,precision_train,precision_valid,precision_test
model_1,0.390793,0.070777,0.083121


## Модель второго уровня

### Подготовка данных для модели второго уровня - обучающая выборка

In [29]:
# Вывод результатов прогноза моделей первого уровня на валидационной выборке.
result_valid.head(2)

,user_id,actual,model_1
0,1,"[853529, 865456, 867607, 872137, 875240, 87737...","[981760, 1005186, 911878, 962568, 826249, 8623..."
1,2,"[838136, 839656, 861272, 866211, 870791, 87391...","[981760, 1005186, 844165, 911878, 826249, 8623..."


In [30]:
# Подготовка таблицы для проверки модели второго уровня.
X_train = utils.prepare_result_lvl_2(data=result_valid, feature_predicted='model_1')

# Проверка.
X_train.head()

,user_id,item_id,actual
0,1,981760,0
1,1,1005186,1
2,1,844165,0
3,1,911878,0
4,1,826249,0


#### Добавление признаков пользователей и товаров

In [31]:
def add_features(X):
    # Добавление признаков пользователей "age_desc" и "income_desc".
    X = X.merge(user_features[['user_id', 'age_desc', 'income_desc']], on='user_id')
    
    # Добавление признаков товаров "manufacturer" и "department".
    X = X.merge(item_features[['item_id', 'manufacturer', 'department']], on='item_id')
    
    # Формирование таблицы признаков пользователь-товар.
    user_item_features = data_train.copy()

    user_item_features['user_item_id'] = (
        user_item_features['user_id']
        .astype(str)
        .str
        .cat(user_item_features['item_id'].astype(str), sep='_')
    )

    user_item_features = (
        user_item_features
        .groupby(by='user_item_id')[['quantity', 'sales_value']]
        .sum()
        .reset_index()
    )
    
    # Формирование идентификатора пользователь-товар.
    X['user_item_id'] = (
        X['user_id']
        .astype(str)
        .str
        .cat(X['item_id'].astype(str), sep='_')
    )
    
    # Добавление признаков пользователь-товар "quantity" и "sales_value".
    X = X.merge(user_item_features, on='user_item_id')
    X = X.drop(columns=['user_item_id'])
    
    return X

In [32]:
# Добавление признаков пользователей и товаров.
X_train = add_features(X_train)

In [33]:
# Проверка структуры сформированной таблицы.
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14449 entries, 0 to 14448
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   user_id       14449 non-null  int64  
 1   item_id       14449 non-null  int64  
 2   actual        14449 non-null  int32  
 3   age_desc      14449 non-null  object 
 4   income_desc   14449 non-null  object 
 5   manufacturer  14449 non-null  int64  
 6   department    14449 non-null  object 
 7   quantity      14449 non-null  int64  
 8   sales_value   14449 non-null  float64
dtypes: float64(1), int32(1), int64(4), object(3)
memory usage: 1.0+ MB


### Подготовка данных для модели второго уровня - тестовая выборка

In [34]:
# Вывод результатов прогноза моделей первого уровня на тестовой выборке.
result_test.head(2)

,user_id,actual,model_1
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[981760, 1005186, 911878, 962568, 826249, 8623..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[981760, 6533889, 1053690, 929668, 844165, 965..."


In [35]:
# Подготовка таблицы для проверки модели второго уровня.
X_test = utils.prepare_result_lvl_2(data=result_test, feature_predicted='model_1')

# Проверка.
X_test.head()

,user_id,item_id,actual
0,1,981760,0
1,1,6533889,0
2,1,1053690,0
3,1,929668,0
4,1,844165,0


#### Добавление признаков пользователей и товаров

In [36]:
# Добавление признаков пользователей и товаров.
X_test = add_features(X_test)

In [37]:
# Проверка структуры сформированной таблицы.
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13047 entries, 0 to 13046
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   user_id       13047 non-null  int64  
 1   item_id       13047 non-null  int64  
 2   actual        13047 non-null  int32  
 3   age_desc      13047 non-null  object 
 4   income_desc   13047 non-null  object 
 5   manufacturer  13047 non-null  int64  
 6   department    13047 non-null  object 
 7   quantity      13047 non-null  int64  
 8   sales_value   13047 non-null  float64
dtypes: float64(1), int32(1), int64(4), object(3)
memory usage: 968.3+ KB


### Построение модели второго уровня

In [38]:
%%time

# Список категориальных признаков.
cat_feats = ['user_id', 'item_id', 'age_desc', 'income_desc', 'department']

# Инициализация модели второго уровня.
model_2 = CatBoostClassifier(silent=True, task_type='GPU', cat_features=cat_feats, random_state=random_state_global)

# Обучение модели второго уровня.
model_2.fit(X_train.drop(columns=['actual']), X_train['actual'])

Wall time: 41.1 s


#### Прогноз на обучающей выборке

In [39]:
def model_2_predict(result, X):
    # Прогноз покупки пользователем товара.
    y_pred = X[['user_id', 'item_id']].copy()
    y_pred['predict'] = model_2.predict(X.drop(columns=['actual']))
    
    # Формирование списка потенциально купленных товаров для каждого пользователя.
    result = result_test.merge(y_pred[y_pred['predict'] == 1]
                               .groupby(by='user_id')['item_id']
                               .agg(list)
                               .reset_index()
                               .rename(columns={'item_id': 'model_2'}),
                               on='user_id',
                               how='left')
    
    # Если модель второго уровня не дала результатов, использовать модель первого уровня.
    result['model_2'] = result['model_2'].fillna(result['model_1'])
    
    # Испольовать для прогноза только первые пять товаров.
    result['model_2'] = result['model_2'].apply(lambda x: x[:5])
    
    return result

In [40]:
# Формирование результатов прогноза двухуровневой модели.
result_valid = model_2_predict(result_valid, X_train)

# Проверка.
result_valid.head()

,user_id,actual,model_1,model_2
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[981760, 1005186, 911878, 962568, 826249, 8623...","[840361, 995242, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[981760, 6533889, 1053690, 929668, 844165, 965...","[981760, 6533889, 1053690, 929668, 844165]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[981760, 6533889, 1005186, 911878, 962568, 961...","[981760, 6533889, 1005186, 911878, 962568]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[981760, 1005186, 844165, 911878, 826249, 8623...",[1082185]
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[981760, 6533889, 1005186, 1053690, 929668, 82...",[1082185]


In [41]:
# Вычисление метрик на валидационной выборке.
df_metrics.loc['model_2', 'precision_valid'] = metrics.precision_at_k(result_valid['actual'], result_valid['model_2'], K=5)

#### Прогноз на тестовой выборке

In [42]:
# Формирование результатов прогноза двухуровневой модели.
result_test = model_2_predict(result_test, X_test)

# Проверка.
result_test.head()

,user_id,actual,model_1,model_2
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[981760, 1005186, 911878, 962568, 826249, 8623...","[995242, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[981760, 6533889, 1053690, 929668, 844165, 965...","[981760, 6533889, 1053690, 929668, 844165]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[981760, 6533889, 1005186, 911878, 962568, 961...","[981760, 6533889, 1005186, 911878, 962568]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[981760, 1005186, 844165, 911878, 826249, 8623...",[1082185]
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[981760, 6533889, 1005186, 1053690, 929668, 82...",[1082185]


In [43]:
# Вычисление метрик на тестовой выборке.
df_metrics.loc['model_2', 'precision_test'] = metrics.precision_at_k(result_test['actual'], result_test['model_2'], K=5)

#### Анализ метрик

In [44]:
# Вывод метрик.
df_metrics

,precision_train,precision_valid,precision_test
model_1,0.390793,0.070777,0.083121
model_2,NaN,0.208342,0.198839


### Сохранение результатов

In [45]:
# Сохранение результатов.
result_test[['user_id', 'model_2']].to_csv(PATH_RESULT, index=False)